최고 점수 튜닝

빠른 학습을 위한 image -> npy 형태로 학습

## Import

In [ ]:
import random
import pandas as pd
import numpy as np
import os
import cv2
from matplotlib import pyplot as plt
from glob import glob
import torch
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm import tqdm
import timm
from torchsummaryX import summary
from sklearn.model_selection import train_test_split, StratifiedKFold

from torch.utils.data import Dataset
from torch import nn

## Config

In [ ]:
from dotenv import load_dotenv
from setup import get_package_root_path
from src.global_exception_handler.v1 import GlobalExceptionHandler
from src.webhook.v1 import TeamsWebhook

load_dotenv()

pakage_name = os.environ.get("PACKAGE_NAME")
root_path = get_package_root_path()

# 웹훅 알림 url (없으면 빈 문자열)
webhook_url = os.environ.get("WEBHOOK_URL")
webhook = TeamsWebhook(webhook_url)

# 핸들링할 예외 종류
except_tuple = (Exception,)
GlobalExceptionHandler(except_tuple=except_tuple, sender=webhook, name="dacon_cars")

In [ ]:
!nvidia-smi

## Hyperparameter Setting

In [ ]:
class CFG:
    root_path = root_path
    # Job Id (보통 파일명과 동일하게)
    job_id = "CLASSICIATION_5_submit_special_spark"

    # 원천 데이터 경로
    data_path = f"{root_path}/data/cars"

    # 학습의 결과물이 저장될 경로
    outputs_path = f"{root_path}/outputs/{job_id}"
    predict_dir = f"{outputs_path}/predict"
    recorder_dir = f"{outputs_path}/recorder"

    learning_late = 0.0001
    batch_size = 32
    epoch = 60
    num_classes = 34

    classes = [
        "chevrolet_malibu_sedan_2012_2016",
        "chevrolet_malibu_sedan_2017_2019",
        "chevrolet_spark_hatchback_2016_2021",
        "chevrolet_trailblazer_suv_2021_",
        "chevrolet_trax_suv_2017_2019",
        "genesis_g80_sedan_2016_2020",
        "genesis_g80_sedan_2021_",
        "genesis_gv80_suv_2020_",
        "hyundai_avante_sedan_2011_2015",
        "hyundai_avante_sedan_2020_",
        "hyundai_grandeur_sedan_2011_2016",
        "hyundai_grandstarex_van_2018_2020",
        "hyundai_ioniq_hatchback_2016_2019",
        "hyundai_sonata_sedan_2004_2009",
        "hyundai_sonata_sedan_2010_2014",
        "hyundai_sonata_sedan_2019_2020",
        "kia_carnival_van_2015_2020",
        "kia_carnival_van_2021_",
        "kia_k5_sedan_2010_2015",
        "kia_k5_sedan_2020_",
        "kia_k7_sedan_2016_2020",
        "kia_mohave_suv_2020_",
        "kia_morning_hatchback_2004_2010",
        "kia_morning_hatchback_2011_2016",
        "kia_ray_hatchback_2012_2017",
        "kia_sorrento_suv_2015_2019",
        "kia_sorrento_suv_2020_",
        "kia_soul_suv_2014_2018",
        "kia_sportage_suv_2016_2020",
        "kia_stonic_suv_2017_2019",
        "renault_sm3_sedan_2015_2018",
        "renault_xm3_suv_2020_",
        "ssangyong_korando_suv_2019_2020",
        "ssangyong_tivoli_suv_2016_2020",
    ]


CFG.__dict__

## CustomDataset

In [ ]:
from sklearn.utils.class_weight import compute_class_weight


class CustomDataset(Dataset):
    def __init__(self, X, y, transforms=None, num_classes: int = None):
        self.X = X
        self.y = y
        self.transforms = transforms
        self.num_classes = num_classes

    def to_categorical(self, y, num_classes):
        """1-hot encodes a tensor"""
        return np.eye(num_classes, dtype="uint8")[y]

    def get_class_weight(self):
        return torch.Tensor(
            compute_class_weight(
                class_weight="balanced", classes=np.unique(self.y), y=self.y
            )
        )

    def __getitem__(self, index):
        img_path = self.X[index]
        image = cv2.cvtColor(np.load(img_path), cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=image)["image"]

        if self.y is None:  # if test
            return image, img_path

        # train or valid
        label = self.y[index]
        if self.num_classes is None:
            return image, label, img_path
        else:
            return image, self.to_categorical(label, self.num_classes), img_path

    def __len__(self):
        return len(self.X)

## Transforms

In [ ]:
train_transform = A.Compose(
    [
        A.PadIfNeeded(
            456,
            456,
            border_mode=0,
            value=(0, 0, 0),
        ),
        A.ShiftScaleRotate(
            shift_limit=0,
            rotate_limit=7,
            border_mode=0,
            value=(0, 0, 0),
            p=0.5,
        ),
        A.ToGray(p=1),
        A.HorizontalFlip(),
        A.OneOf(
            [
                A.GaussianBlur(blur_limit=(3, 25), p=1),
                A.GaussNoise(p=1),
                A.Sharpen(p=1),
                A.Equalize(p=1),
            ],
            p=0.9,
        ),
        A.HueSaturationValue(val_shift_limit=10, p=0.9),
        A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=1),
        A.OneOf(
            [
                A.CoarseDropout(fill_value=255, max_height=25, max_width=25),
                A.CoarseDropout(fill_value=128, max_height=25, max_width=25),
                A.CoarseDropout(fill_value=0, max_height=25, max_width=25),
            ],
            p=1,
        ),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), max_pixel_value=255.0),
        ToTensorV2(),
    ]
)


val_transform = A.Compose(
    [
        A.PadIfNeeded(
            456,
            456,
            border_mode=0,
            value=(0, 0, 0),
        ),
        A.ToGray(p=1),
        # A.GaussianBlur(blur_limit=(3, 21), p=1),
        # A.HueSaturationValue(),
        # A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=1),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), max_pixel_value=255.0),
        ToTensorV2(),
    ]
)

## Init dataset

In [ ]:
data = None
for i, cls in enumerate(range(CFG.num_classes)):
    data_path_list = sorted(glob(f"{CFG.data_path}/{cls}/*.npy"))
    data_path_list = np.expand_dims(np.array(data_path_list), 1)

    labels = np.ones(data_path_list.shape, dtype=np.uint8) * i

    temp = np.concatenate([data_path_list, labels], axis=1)

    data = temp if data is None else np.concatenate([data, temp], axis=0)

In [ ]:
df = pd.DataFrame(data)
df.columns = ["path", "label"]
df = df.astype({"path": "string", "label": "int"})
df

In [ ]:
for i, cls in enumerate(range(CFG.num_classes)):
    print(f'{cls} : {df[df["label"] == i].shape[0]}')

print("")
print(f"전체 : {df.shape[0]}")

## Train / Validation Split

In [ ]:
data_fold_splitter = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
checker = data_fold_splitter.get_n_splits(X=df, y=df["label"])
print(checker)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score


def save_pred(saved_path, path, y_true, y_pred, y_prob=None):
    os.makedirs(saved_path)

    df_data = [path, y_true, y_pred]
    df_columns = ["path", "y_true", "y_pred"]

    if y_prob != None:
        df_data.append(y_prob)
        df_columns.append("y_prob")

    df = pd.DataFrame(np.array(df_data).T)
    df.columns = df_columns

    df.to_csv(f"{saved_path}/pred.csv", index=False)

    ### 임시 confusion_matrix
    cm = confusion_matrix(y_true, y_pred)
    sns.set(rc={"figure.figsize": (21, 21)})
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens")
    _val_score = f1_score(y_true, y_pred, average="macro")
    plt.xlabel(f"Pred / F1-score: {_val_score:.3f}")
    plt.ylabel("Real")

    classes_point = list(map(lambda x: x + 0.5, range(CFG.num_classes)))
    classes = list(range(CFG.num_classes))
    plt.xticks(classes_point, classes)
    plt.yticks(classes_point, classes)
    plt.savefig(f"{saved_path}/c_matrix.jpg")
    plt.clf()

## Valid

In [ ]:
from src.image_eda.v1 import tensor2im
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    cohen_kappa_score,
    confusion_matrix,
)


def valid(model, criterion, data_loader, device):
    model.eval()
    val_loss = []

    epoch_paths = []
    y_true = []
    y_pred = []
    # y_probs = []
    with torch.no_grad():
        for batch_index, (images, labels, paths) in enumerate(tqdm(data_loader)):
            if batch_index % 10 == 0:
                temp_img = images[0].cpu().detach()
                temp_img = tensor2im(temp_img)

                cv2.imwrite(f"{CFG.root_path}/temp/valid_img.jpg", temp_img)

            images = images.to(device, dtype=torch.float)
            labels = labels.to(device, dtype=torch.float)

            probs = model(images)
            loss = criterion(probs, labels)

            probs = probs.cpu().detach().numpy()
            labels = labels.cpu().detach().numpy()

            preds = np.argmax(probs, 1).astype(np.uint8)
            labels = np.argmax(labels, 1).astype(np.uint8)

            preds = preds.flatten()
            labels = labels.flatten()

            y_pred += preds.tolist()
            y_true += labels.tolist()
            # y_probs += probs.tolist()
            epoch_paths += paths

            val_loss.append(loss.item())

    val_loss = np.mean(val_loss)
    val_score = accuracy_score(y_true, y_pred)

    return {
        "val_loss": val_loss,
        "val_score": val_score,
        "path": epoch_paths,
        "y_true": y_true,
        "y_pred": y_pred,
    }

## Train

In [ ]:
def train(model, criterion, optimizer, data_loader, device, grad_scaler=None):
    model.train()
    train_loss = []

    epoch_paths = []
    y_true = []
    y_pred = []
    for batch_index, (images, labels, paths) in enumerate(tqdm(data_loader)):
        if batch_index % 10 == 0:
            temp_img = images[0].detach().cpu()
            temp_img = tensor2im(temp_img)

            cv2.imwrite(f"{CFG.root_path}/temp/train_img.jpg", temp_img)

        images = images.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.float)

        optimizer.zero_grad()
        if grad_scaler is None:
            probs = model(images)
            loss = criterion(probs, labels)
            loss.backward()
            optimizer.step()
        else:
            with torch.cuda.amp.autocast():
                probs = model(images)
                loss = criterion(probs, labels)

            grad_scaler.scale(loss).backward()
            grad_scaler.step(optimizer)
            grad_scaler.update()

        probs = probs.cpu().detach().numpy()
        labels = labels.cpu().detach().numpy()

        preds = np.argmax(probs, 1).astype(np.uint8)
        labels = np.argmax(labels, 1).astype(np.uint8)

        preds = preds.flatten()
        labels = labels.flatten()

        y_pred += preds.tolist()
        y_true += labels.tolist()
        epoch_paths += paths

        train_loss.append(loss.item())

    train_loss = np.mean(train_loss)
    train_score = accuracy_score(y_true, y_pred)

    return {
        "train_loss": train_loss,
        "train_score": train_score,
    }

## Model Define

In [ ]:
def create_model(num_classes: int) -> nn.Module:
    # model = models.efficientnet_b5(weights=models.EfficientNet_B5_Weights.DEFAULT)
    # model.classifier = nn.Sequential(nn.Linear(2048, num_classes), nn.Softmax())

    # model = timm.models.eva.eva02_base_patch14_448(pretrained=True)
    # model.head = nn.Sequential(
    #     nn.Linear(768, ),
    #     nn.BatchNorm1d(),
    #     nn.GELU(),
    #     nn.Dropout(0.5),
    #     nn.Linear(768, num_classes),
    # )

    # model = models.convnext_large(weights=models.ConvNeXt_Large_Weights.DEFAULT)
    # model.classifier[2] = nn.Sequential(
    #     nn.Linear(1536, 1024),
    #     nn.LayerNorm(1024),
    #     nn.SiLU(),
    #     nn.Dropout1d(0.5),
    #     nn.Linear(1024, num_classes),
    # )

    model = timm.models.convnext.convnext_large(pretrained=True)
    model.head.fc = nn.Sequential(
        nn.Linear(1536, 768),
        nn.LayerNorm(768),
        nn.SiLU(),
        nn.Dropout(0.5),
        nn.Linear(768, num_classes),
    )

    return model


create_model(34)

## Snapshot Notebook

In [ ]:
import shutil

try:
    import IPython

    notebook_path = IPython.extract_module_locals()[1]["__vsc_ipynb_file__"]
except:
    notebook_path = f"{os.getcwd()}/{CFG.job_id}.ipynb"


os.makedirs(CFG.outputs_path, exist_ok=True)
shutil.copy(notebook_path, f"{CFG.outputs_path}/{os.path.split(notebook_path)[1]}")

## Run!!

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

In [ ]:
from src.random_seed.v1 import seed_everything, seed_worker

seed_everything(0)

In [ ]:
grad_scaler = torch.cuda.amp.GradScaler()

In [ ]:
from src.recorder.v1 import Recorder
from time import time


for fold_index, (train_idx, valid_idx) in enumerate(
    data_fold_splitter.split(X=df, y=df["label"])
):
    train_df = df.iloc[train_idx]
    val_df = df.iloc[valid_idx]

    train_dataset = CustomDataset(
        train_df["path"].values,
        train_df["label"].values,
        train_transform,
        CFG.num_classes,
    )
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=16,  #
        pin_memory=True,
        drop_last=False,
        worker_init_fn=seed_worker,
    )
    #
    val_dataset = CustomDataset(
        val_df["path"].values, val_df["label"].values, val_transform, CFG.num_classes
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=CFG.batch_size,
        shuffle=True,
        num_workers=16,
        pin_memory=True,
        drop_last=False,
        worker_init_fn=seed_worker,
    )
    model = create_model(CFG.num_classes)
    model.to(device)
    model.cuda()
    optimizer = torch.optim.AdamW(params=model.parameters(), lr=CFG.learning_late)
    scheduler = None

    recorder = Recorder(
        f"{CFG.recorder_dir}/fold_{fold_index}", model, optimizer, scheduler
    )
    print(f"fold_{fold_index} start")
    if recorder.load_checkpoint(device, "checkpoint.pt"):
        print(f"loaded current_epoch: {recorder.current_epoch}")

    criterion = nn.CrossEntropyLoss()

    best_val_loss = 100
    for epoch_index in range(recorder.current_epoch, CFG.epoch):
        seed_everything(epoch_index)

        train_start_timestamp = time()
        train_dict = train(
            model, criterion, optimizer, train_loader, device, grad_scaler
        )
        train_elapsed_time = time() - train_start_timestamp

        val_start_timestamp = time()
        val_dict = valid(model, criterion, val_loader, device)
        val_elapsed_time = time() - val_start_timestamp

        recorder.update_row_dict("epoch", epoch_index + 1)

        recorder.update_row_dict("train_loss", train_dict["train_loss"])
        recorder.update_row_dict("val_loss", val_dict["val_loss"])

        recorder.update_row_dict("train_score", train_dict["train_score"])
        recorder.update_row_dict("val_score", val_dict["val_score"])

        recorder.update_row_dict("train_elapsed_time", train_elapsed_time)
        recorder.update_row_dict("val_elapsed_time", val_elapsed_time)
        recorder.flush_row_dict(is_print=True)
        recorder.save_line_plot(["loss"], [0, 0.1])

        save_pred(
            f"{CFG.predict_dir}/fold_{fold_index}/{epoch_index+1}",
            val_dict["path"],
            val_dict["y_true"],
            val_dict["y_pred"],
        )

        if recorder.is_best_score(val_dict["val_loss"], "min"):
            print(f"best epoch: {epoch_index + 1}")
            recorder.save_checkpoint(epoch_index, "best_model.pt")

        recorder.save_checkpoint(epoch_index)

## Test

In [ ]:
def get_smaller_box_with_iou_threshold(box1, box2, iou_threshold=0.1):
    # box1 = (x1, y1, x2, y2)
    # box2 = (x1, y1, x2, y2)

    # 상자1의 좌표 추출
    x1_box1, y1_box1, x2_box1, y2_box1 = box1
    # 상자2의 좌표 추출
    x1_box2, y1_box2, x2_box2, y2_box2 = box2

    # 상자1의 면적 계산
    area_box1 = (x2_box1 - x1_box1) * (y2_box1 - y1_box1)
    # 상자2의 면적 계산
    area_box2 = (x2_box2 - x1_box2) * (y2_box2 - y1_box2)

    # 상자1과 상자2의 겹치는 영역 계산
    x_left = max(x1_box1, x1_box2)
    y_top = max(y1_box1, y1_box2)
    x_right = min(x2_box1, x2_box2)
    y_bottom = min(y2_box1, y2_box2)

    # 겹치는 영역의 면적 계산
    intersection_area = max(0, x_right - x_left) * max(0, y_bottom - y_top)

    # IoU 계산
    iou = intersection_area / float(area_box1 + area_box2 - intersection_area)

    # IoU가 지정한 임계값 이상인 경우, 더 작은 상자 index를 반환
    if iou >= iou_threshold:
        if area_box1 < area_box2:
            return 0
        else:
            return 1
    else:
        return None

In [ ]:
print(get_smaller_box_with_iou_threshold((0, 0, 9, 9), (2, 2, 5, 5)))

In [ ]:
test_transform = A.Compose(
    [
        A.PadIfNeeded(
            456,
            456,
            border_mode=0,
            value=(0, 0, 0),
        ),
        # A.RandomBrightnessContrast(p=1),
        A.ToGray(p=1),
        A.Equalize(by_channels=False, p=1),
        A.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5), max_pixel_value=255.0),
        ToTensorV2(),
    ]
)

In [ ]:
model_list = []
import ttach as tta

tta_transforms = tta.Compose(
    [
        # tta.FiveCrops(350, 350),
        # tta.Rotate90(angles=[0, 180]),
        # tta.Scale(scales=[1, 2, 4]),
        # tta.Multiply(factors=[0.9, 1, 1.1]),
        tta.HorizontalFlip(),
    ]
)

for i in range(0, 5):
    model = create_model(CFG.num_classes)
    model.to(device)
    model.cuda()

    check_point = torch.load(
        f"{CFG.recorder_dir}/fold_{i}/best_model.pt",
        map_location=device,
    )
    print(check_point["epoch"])
    print(check_point["best_score"])
    model.load_state_dict(check_point["model"])
    model.eval()
    tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)
    model_list.append(tta_model)

# for i in range(0, 5):
#     model = create_model(CFG.num_classes)
#     model.to(device)
#     model.cuda()

#     check_point = torch.load(
#         f"{CFG.recorder_dir}/fold_{i}/checkpoint.pt",
#         map_location=device,
#     )
#     print(check_point["epoch"])
#     print(check_point["best_score"])
#     model.load_state_dict(check_point["model"])
#     model.eval()
#     tta_model = tta.ClassificationTTAWrapper(model, tta_transforms)
#     model_list.append(tta_model)

In [ ]:
test_index = 0  # 656

In [ ]:
from torch.nn import functional as F


test_list = sorted(glob("/data/dacon_cars/data/test_cars/*.npy"))
# test_list = sorted(glob("/data/dacon_cars/data/test_cars/155231127_0.npy"))
print(test_list[test_index])
print(test_index)
img = cv2.cvtColor(np.load(test_list[test_index]), cv2.COLOR_BGR2RGB)
test_index += 1
with torch.no_grad():
    print(img.shape)
    img = img[5:-5, 5:-5]
    test_img = test_transform(image=img)["image"]
    test_img = torch.Tensor(test_img).to(device, dtype=torch.float)
    test_img = torch.unsqueeze(test_img, 0)
    ensemble_probs = None
    for tta_model in model_list:
        probs = F.softmax(tta_model(test_img), dim=1)
        probs = np.round(probs.cpu().detach().numpy(), 3)

        if ensemble_probs is None:
            ensemble_probs = probs
        else:
            ensemble_probs = ensemble_probs + probs

    ensemble_probs = np.round(ensemble_probs / len(model_list), 3)

    top1 = np.max(ensemble_probs)
    label = np.argmax(ensemble_probs)
    spark_score = ensemble_probs[0][4] + ensemble_probs[0][12] + ensemble_probs[0][23]

    gene = ensemble_probs[0][6] + ensemble_probs[0][7]

    print(ensemble_probs)
    print(top1)
    print(f"spark: {spark_score}")
    print(f"gene: {gene}")
    print(label)
    print(CFG.classes[label])
    plt.figure(figsize=(5, 5))
    plt.imshow(tensor2im(test_img[0]))
    # f"{test_path}/test"

In [ ]:
bbox_submit = pd.read_csv("/data/dacon_cars/data/bbox_submit_npy.csv")
filenames = sorted(os.listdir("/data/dacon_cars/data/test"))

total_list = []
for target in tqdm(iter(filenames)):
    car_info_df = bbox_submit[
        (bbox_submit["file_name"] == target)
        # & ((bbox_submit["point3_x"] - bbox_submit["point1_x"]) <= 370)
        # & ((bbox_submit["point3_y"] - bbox_submit["point1_y"]) <= 441)
    ]
    # print(one_test)
    # if "064507368" not in target:
    #     continue

    infer_list = []
    for i, car_info in car_info_df.iterrows():
        file_name = car_info["file_name"]

        img = cv2.cvtColor(
            cv2.imread(f"/data/dacon_cars/data/test/{file_name}"),
            cv2.COLOR_BGR2RGB,
        )
        pt1x = int(np.round(car_info["point1_x"])) + 5
        pt1y = int(np.round(car_info["point1_y"])) + 5
        pt2x = int(np.round(car_info["point2_x"])) - 5
        pt2y = int(np.round(car_info["point2_y"])) + 5
        pt3x = int(np.round(car_info["point3_x"])) - 5
        pt3y = int(np.round(car_info["point3_y"])) - 5
        pt4x = int(np.round(car_info["point4_x"])) + 5
        pt4y = int(np.round(car_info["point4_y"])) - 5

        img = img[pt1y:pt3y, pt1x:pt3x]
        with torch.no_grad():
            img = test_transform(image=img)["image"]
            img = torch.Tensor(img).to(device, dtype=torch.float)
            img = torch.unsqueeze(img, 0)
            ensemble_probs = None
            for tta_model in model_list:
                probs = F.softmax(tta_model(img), dim=1)
                probs = np.round(probs.cpu().detach().numpy(), 3)

                if ensemble_probs is None:
                    ensemble_probs = probs
                else:
                    ensemble_probs = ensemble_probs + probs

            ensemble_probs = np.round(ensemble_probs / len(model_list), 3)

        # for j in range(len(ensemble_probs)):
        #     ensemble_probs[j][2] = (
        #         ensemble_probs[j][4] + ensemble_probs[j][12] + ensemble_probs[j][23]
        #     )
        # if "075817746" in file_name:
        #     print(file_name)
        class_id = np.argmax(ensemble_probs)
        confidence = ensemble_probs[0][class_id]
        spark_score = (
            ensemble_probs[0][4] + ensemble_probs[0][12] + ensemble_probs[0][23]
        )

        infer_list.append(
            [
                file_name,
                class_id,
                confidence,
                pt1x,
                pt1y,
                pt2x,
                pt2y,
                pt3x,
                pt3y,
                pt4x,
                pt4y,
                spark_score,
            ]
        )

    infer_array = np.array(infer_list)
    max_confidence_list = []
    try:
        # if len(np.where(infer_array.T[2].astype(float) > 0.99)[0]) > 1:
        high_confi = np.where(infer_array.T[2].astype(float) >= 0.999)[0]
        if len(high_confi) > 1:
            print(target)
            print(infer_array)

            box1 = infer_array[high_confi][0]
            box2 = infer_array[high_confi][1]

            smaller_index = get_smaller_box_with_iou_threshold(
                (int(box1[3]), int(box1[4]), int(box1[7]), int(box1[8])),
                (int(box2[3]), int(box2[4]), int(box2[7]), int(box2[8])),
            )
            print(smaller_index)

            if smaller_index is None:  # 안겹침
                total_list.append(infer_array[np.argmax(infer_array.T[2])].tolist())
                # total_list.append(box1.tolist())
                # total_list.append(box2.tolist())
            else:
                if smaller_index == 0:
                    total_list.append(box1.tolist())
                else:
                    total_list.append(box2.tolist())
            continue

        top_score = float(infer_array[np.argmax(infer_array.T[2])][2])
        if top_score < 0.65:
            new_spark = infer_array[np.argmax(infer_array.T[11])].tolist()

            if top_score <= float(new_spark[11]):
                new_spark[1] = 2
                total_list.append(new_spark)
                print(target)
                print(f"new_spark!: {new_spark}")
                continue

        total_list.append(infer_array[np.argmax(infer_array.T[2])].tolist())

    except Exception as e:
        print(e)
        print(car_info)

    # total_list.append(max_confidence_list.copy())

In [ ]:
submit_df = pd.DataFrame(
    total_list,
    columns=[
        "file_name",
        "class_id",
        "confidence",
        "point1_x",
        "point1_y",
        "point2_x",
        "point2_y",
        "point3_x",
        "point3_y",
        "point4_x",
        "point4_y",
        "spark_score",
    ],
)

# submit_df.astype(
#     {
#         "file_name": "string",
#         "class_id": "int",
#         "confidence": "float",
#         "point1_x": "float",
#         "point1_y": "float",
#         "point2_x": "float",
#         "point2_y": "float",
#         "point3_x": "float",
#         "point3_y": "float",
#         "point4_x": "float",
#         "point4_y": "float",
#     }
# )
submit_df

In [ ]:
submit_df.to_csv("/data/submit_spark_score_단일_보정_재현1.csv", index=False)
submit_df = pd.read_csv("/data/submit_spark_score_단일_보정_재현1.csv")

In [ ]:
img = cv2.imread(f"/data/dacon_cars/data/test/134457909.png")
img = img[5:-5, 5:-5]
img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
print(img.shape)
plt.imshow(img)

In [ ]:
submit_df[(submit_df["file_name"] == "161544056.png")]

In [ ]:
submit_df[(submit_df["class_id"] == 2)]  #  & (submit_df["confidence"] < 0.7)

In [ ]:
i = 0

In [ ]:
submit_df = pd.read_csv("/data/submit_spark_score_단일_보정_재현1.csv")

# data = submit_df[
#     (submit_df["class_id"] == 23)
#     & (submit_df["confidence"] < 0.999)
#     & (submit_df["confidence"] > 0.99)
# ].iloc[i]

# file_name = data["file_name"]
# score = data["confidence"]
# i += 1

# print(score)
# img = cv2.imread(f"/data/dacon_cars/data/test/{file_name}")
# img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
# plt.figure(figsize=(15, 15))
# plt.imshow(img)


submit_df.loc[
    submit_df[
        (submit_df["class_id"] == 12) & (submit_df["confidence"] < 0.65)
    ].index.to_list(),
    "class_id",
] = 2

submit_df.loc[
    submit_df[
        (submit_df["class_id"] == 4) & (submit_df["confidence"] < 0.65)
    ].index.to_list(),
    "class_id",
] = 2

submit_df.loc[
    submit_df[
        (submit_df["class_id"] == 8) & (submit_df["confidence"] < 0.65)
    ].index.to_list(),
    "class_id",
] = 2

submit_df.loc[
    submit_df[
        (submit_df["class_id"] == 22) & (submit_df["confidence"] < 0.65)
    ].index.to_list(),
    "class_id",
] = 2

submit_df.loc[
    submit_df[
        (submit_df["class_id"] == 27) & (submit_df["confidence"] < 0.65)
    ].index.to_list(),
    "class_id",
] = 2

submit_df.loc[
    submit_df[
        (submit_df["class_id"] == 23) & (submit_df["confidence"] < 0.99)
    ].index.to_list(),
    "class_id",
] = 2


# submit_df.loc[
#     submit_df[
#         (submit_df["confidence"] < 0.7)
#         &(submit_df["class_id"] == 4)
#         &(submit_df["class_id"] == 8)
#         &(submit_df["class_id"] == 11)
#         &(submit_df["class_id"] == 15)
#         &(submit_df["class_id"] == 20)
#         &(submit_df["class_id"] == 27)
#         &(submit_df["class_id"] == 33)
#     ].index.to_list(),
#     "class_id",
# ] = 2

# submit_df.loc[
#     submit_df[
#         (submit_df["confidence"] < 0.999)
#         &(submit_df["class_id"] == 23)
#     ].index.to_list(),
#     "class_id",
# ] = 2

# 12 빠짐
# # 26 < 0.5
# # 4, 11, 8, 15, 20, 27, 33 < 0.7
# # 23 < 0.99

submit_df.to_csv("/data/submit_spark_score_단일_보정_재현2.csv", index=False)

In [ ]:
from torch.nn import functional as F


# test_list = sorted(glob("/data/dacon_cars/data/test_cars/*.npy"))
test_list = sorted(glob("/data/dacon_cars/data/test_cars/155231127_1.npy"))
print(test_list[test_index])
print(test_index)
img = cv2.cvtColor(np.load(test_list[test_index]), cv2.COLOR_BGR2RGB)
# test_index += 1
with torch.no_grad():
    print(img.shape)
    img = img[5:-5, 5:-5]
    test_img = test_transform(image=img)["image"]
    test_img = torch.Tensor(test_img).to(device, dtype=torch.float)
    test_img = torch.unsqueeze(test_img, 0)
    ensemble_probs = None
    for tta_model in model_list:
        probs = F.softmax(tta_model(test_img), dim=1)
        probs = np.round(probs.cpu().detach().numpy(), 3)

        if ensemble_probs is None:
            ensemble_probs = probs
        else:
            ensemble_probs = ensemble_probs + probs

    ensemble_probs = np.round(ensemble_probs / len(model_list), 3)

    top1 = np.max(ensemble_probs)
    label = np.argmax(ensemble_probs)
    spark_score = ensemble_probs[0][4] + ensemble_probs[0][12] + ensemble_probs[0][23]

    gene = ensemble_probs[0][6] + ensemble_probs[0][7]

    print(ensemble_probs)
    print(top1)
    print(f"spark: {spark_score}")
    print(f"gene: {gene}")
    print(label)
    print(CFG.classes[label])
    plt.figure(figsize=(5, 5))
    plt.imshow(tensor2im(test_img[0]))
    # f"{test_path}/test"

In [ ]:
submit_df = pd.read_csv("/data/submit_special_spark.csv").iloc[:, 1:]
submit_df

from src.image_eda.v1 import apply_bbox

for i, row in tqdm(submit_df.iterrows()):
    img = cv2.imread(f"/data/dacon_cars/data/test/{row['file_name']}")

    img = apply_bbox(
        img,
        [row["class_id"] + 1],
        [[row["point1_x"], row["point1_y"], row["point3_x"], row["point3_y"]]],
        CFG.classes,
        [row["confidence"]],
        [0, 255, 0],
    )

    cv2.imwrite(
        f"/data/dacon_cars/data/draw_bbox/{row['file_name'].replace('.png', '.jpg')}",
        img,
    )

In [ ]:
import shutil

for i, row in submit_df[submit_df["class_id"] == 2].iterrows():
    shutil.copy(
        f"/data/dacon_cars/data/test_cars_image/{row['file_name'].replace('.png', '_0.png')}",
        f"/data/dacon_cars/data/export/{row['file_name'].replace('.png', '_0.png')}",
    )

In [ ]:
plt.figure(figsize=(15, 15))
plt.hist(submit_df["class_id"], 34)

In [ ]:
submit2_df = pd.read_csv("/data/submit2.csv")

In [ ]:
compare_df = pd.concat(
    [submit_df["file_name"], submit_df["class_id"], submit2_df["class_id"]],
    axis=1,
)

compare_df = pd.merge(
    submit_df[["file_name", "class_id"]],
    submit2_df[["file_name", "class_id"]],
    how="outer",
    left_index=True,
    right_index=True,
    indicator=True,
)

compare_df.columns = ["file_name", "class_id1", "class_id2"]
compare_df

In [ ]:
print(compare_df.compare(compare_df, align_axis=1))  # index